In [3]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import pandas as pd 
import numpy as np
import os

#import data 
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial
DATA_PATH = "/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial/raw_data/"
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial


'/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial'

In [4]:
from os import listdir
FILE_LIST = listdir(DATA_PATH)
print(FILE_LIST)

#test
#FILE_LIST[0].split(".csv")[0]

df_dic={} #create a dictionary with names of datasets as keys and datasets as values
for i, e in enumerate(FILE_LIST):
  df_dic[str(FILE_LIST[i].split(".csv")[0])]= pd.read_csv("raw_data/" + e)

#test
#df_dic['diaries_transactions_all'].columns

['con_prod_hh_over_trx_yr_mo_usd.csv', 'con_pur_hh_by_trx_yr_mo_usd.csv', 'demo_hh.csv', 'demo_ind.csv', 'diaries_goingson_hh.csv', 'diaries_wellbeing_ind.csv', 'edu_hh.csv', 'edu_ind.csv', 'housing_conditions_ownership.csv', 'inc_hh_by_trx_yr_mo_usd_incRR.csv', 'inc_ind_by_trx_yr_mo_usd_incRR.csv', 'pov_hh_by_trx_yr_mo_usd_incRR.csv', 'rem_hh_by_trx_yr_mo_usd.csv', 'pov_hh_by_trx_yr_mo_usd_excRR.csv', 'diaries_transactions_all.csv']


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
#start merging
#first we merge datasets with same keys together, then we merge the bigger (already merged) datasets together
from functools import reduce

dfs = [df_dic['diaries_transactions_all'], df_dic['demo_hh'] , df_dic['con_prod_hh_over_trx_yr_mo_usd'], df_dic['edu_hh']]

result_hhids = reduce(lambda df_left, df_right: pd.merge(df_left, df_right, 
                                              right_on = 'hh_ids', left_on = 'hh_ids', 
                                              how='outer'), 
                  dfs)

In [ ]:
print(result_hhids)

In [6]:
dfs = [df_dic['con_pur_hh_by_trx_yr_mo_usd'], df_dic['inc_hh_by_trx_yr_mo_usd_incRR'], df_dic['pov_hh_by_trx_yr_mo_usd_excRR'], df_dic['pov_hh_by_trx_yr_mo_usd_incRR'], df_dic['rem_hh_by_trx_yr_mo_usd']]

result_hhids_trxyrmo = reduce(lambda df_left, df_right: pd.merge(df_left, df_right, 
                                              right_on = ['hh_ids', 'trx_yr_mo'], left_on =['hh_ids', 'trx_yr_mo'], 
                                              how= 'outer'),
                  dfs)

In [ ]:
print(result_hhids_trxyrmo)

In [7]:
dfs = [df_dic['diaries_wellbeing_ind'], df_dic['diaries_goingson_hh']]

result_hhids_intdate = reduce(lambda df_left, df_right: pd.merge(df_left, df_right, 
                                              right_on = ['hh_ids', 'int_date'], left_on =['hh_ids', 'int_date'], 
                                              how= 'outer'),
                  dfs)


In [ ]:
print(result_hhids_intdate)

In [8]:
dfs = [df_dic['demo_ind'], df_dic['edu_ind']]

result_hhids_inds = reduce(lambda df_left, df_right: pd.merge(df_left, df_right, 
                                              right_on = ['hh_ids', 'm_ids'], left_on =['hh_ids', 'm_ids'], 
                                              how= 'outer'),
                  dfs)

#convert to string for merging purposes
result_hhids_inds['m_ids'] = result_hhids_inds['m_ids'].astype(str)

In [ ]:
print(result_hhids_inds)

In [9]:
#final part of merging
official_1 = pd.merge(result_hhids, result_hhids_trxyrmo, right_on = ['hh_ids', 'trx_yr_mo'], left_on =['hh_ids', 'trx_yr_mo'], 
                                              how= 'outer')



In [10]:
official_2 = pd.merge(official_1, result_hhids_intdate, right_on = ['hh_ids', 'int_date'], left_on =['hh_ids', 'int_date'], 
                                              how= 'outer')

In [11]:
official= pd.merge(official_2, result_hhids_inds, right_on = ['hh_ids', 'm_ids'], left_on =['hh_ids', 'm_ids_owner'], 
                                              how= 'outer')


In [ ]:
official.head()

In [ ]:
official[official['hh_ids'] == 'KELDK01']

final comment on merging --> DID NOT MANAGE TO MERGE inc_ind_by_trx_yr_mo_usd_incRR and housing_conditions --> remember to include limitations of this choice in our report!

In [ ]:
#replacing NAs consistently along the whole dataset
official.replace('.', np.NaN)
official.replace(' ', np.NaN)

In [60]:
#checking for empty columns 
empty_cols = [col for col in official.columns if official[col].isnull().all()]

print(empty_cols) #there are no empty columns

[]


In [13]:
#dropping unimportant cols 

official = official.drop(columns = ['first_trx_date_hh', 'last_trx_date_hh', 'tot_hh_daysofobs', 'tot_hh_monthsofobs', 'interview_designation', 'int_date', 'int_month', 'int_year', 'int_yr_mo', 'first_int_date', 'unique_accnts', 'unique_hm_owner', 'tot_acc_daysofobs', 'tot_acc_monthsofobs', 'trx_stdtime_days_hh', 'trx_stdtime_mnths_hh', 'trx_stdtime_days_acc', 'trx_stdtime_mnths_acc', 'trx_family_desc', 'trx_class_desc', 'trx_type_desc', 'trx_mode_code', 'trx_place_incommunity', 'trx_distance_km', 'trx_value_kes', 'trx_inkind_units', 'trx_inkind_value_kes'])


In [14]:
#dropping cols with more than 80% of NaNs

na_perc = official[official.columns[official.isnull().any()]].isnull().sum() * 100 / official.shape[0]
#print(na_perc)

nas_list = []
for i in na_perc.keys():
  if na_perc[i] > 80.0:
    nas_list.append(i)

#print(nas_list)

unique_hhs                    99.955018
account_ids                    0.128423
m_ids_owner                    0.128423
account_bsheet_desig          84.658945
account_startclose_balance    98.767138
                                ...    
dem_i_children_tot            64.738961
dem_i_children_inhh           64.724149
edu_i_enrolled                62.268889
edu_i_attain                  62.502497
edu_i_attain_yrs              62.548159
Length: 175, dtype: float64


In [20]:
#DROP MOSTLY EMPTY COLUMNS
official = official.drop(columns = nas_list)

#official.head()

In [25]:
official[official.isnull().any(axis=1)]

,hh_ids,account_ids,m_ids_owner,first_trx_date_acc,last_trx_date_acc,trx_id,m_ids_trx,trx_date,trx_month,trx_year,...,dem_i_age_workage,dem_i_relhead,dem_i_hhead,dem_i_marstat,dem_i_tribe,dem_i_children_tot,dem_i_children_inhh,edu_i_enrolled,edu_i_attain,edu_i_attain_yrs
0,KELDL02,58134738501900000,58134383397900000,02oct2013,02oct2013,1.081381e+17,58134383397900000,02oct2013,10.0,2013.0,...,1.0,Household head,1.0,Married/living together,Luo,2.0,0.0,0.0,Secondary (some or complete),12.0
1,KELDL02,108136895956500000,58134383397900000,18may2013,02oct2013,1.081382e+17,58134383397900000,02oct2013,10.0,2013.0,...,1.0,Household head,1.0,Married/living together,Luo,2.0,0.0,0.0,Secondary (some or complete),12.0
2,KELDL02,58134738868100000,58134383397900000,06nov2012,02oct2013,1.081382e+17,58134383397900000,02oct2013,10.0,2013.0,...,1.0,Household head,1.0,Married/living together,Luo,2.0,0.0,0.0,Secondary (some or complete),12.0
3,KELDL02,58134738791500000,58134383397900000,21sep2013,02oct2013,1.081382e+17,58134383397900000,02oct2013,10.0,2013.0,...,1.0,Household head,1.0,Married/living together,Luo,2.0,0.0,0.0,Secondary (some or complete),12.0
4,KELDL02,58134738197200000,58134383397900000,03sep2012,02oct2013,1.081381e+17,58134383397900000,02oct2013,10.0,2013.0,...,1.0,Household head,1.0,Married/living together,Luo,2.0,0.0,0.0,Secondary (some or complete),12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735844,KVIHK39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,Son or daughter,0.0,Never married/lived together,Luhya,NaN,NaN,1.0,Nursery/ Kindergarten,0.0
735845,KVIHK39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,Son or daughter,0.0,Never married/lived together,Luhya,NaN,NaN,1.0,NaN,NaN
735846,KVIHK40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,Son or daughter,0.0,Never married/lived together,Luhya,NaN,NaN,1.0,Nursery/ Kindergarten,0.0
735847,KVIHK40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,Son or daughter,0.0,Never married/lived together,Luhya,NaN,NaN,1.0,No education,0.0


Merged dataframe is ready to be split into train and test!

In [ ]:
os.getcwd()

'/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial'

In [ ]:
os.makedirs('clean_data', exist_ok=True)  
compression_opts = dict(method='zip',
                        archive_name='official.csv')
official.to_csv('clean_data/official.zip', index=False,
          compression=compression_opts) 